说明：本文档给出了python常用脚本

In [3]:
import pandas as pd
import numpy as np

# 计算数值型列的outliers，辅助正态性检验

In [14]:
df1 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT.xlsx",sheet_name="Sheet1");

In [15]:
df1.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff,MGMPI
0,1,Tracy,28,F,316,Base,10,3.0,73.356009
1,1,Tracy,28,F,316,Week1,15,2.5,66.468254
2,1,Tracy,28,F,316,Week2,25,1.0,64.980159
3,1,Tracy,28,F,316,Week6,55,0.5,83.524397
4,1,Tracy,28,F,316,Week8,60,0.5,73.469388


In [16]:
# 方差分析模型要求水平组合上的数值型变量值符合正态性。本方法将 “1.96*标准差” 范围外
# 的值定义为 outliers，并创建 outliers 列对其进行标记。

newdf1 = df1.copy()
newdf1["outliers"] = abs(newdf1["MGMPI"]-newdf1["MGMPI"].mean()) > 1.96*newdf1["MGMPI"].std()
newdf1.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff,MGMPI,outliers
0,1,Tracy,28,F,316,Base,10,3.0,73.356009,False
1,1,Tracy,28,F,316,Week1,15,2.5,66.468254,False
2,1,Tracy,28,F,316,Week2,25,1.0,64.980159,False
3,1,Tracy,28,F,316,Week6,55,0.5,83.524397,False
4,1,Tracy,28,F,316,Week8,60,0.5,73.469388,False


In [17]:
# 将 outliers 行选出

newdf1.loc[newdf1["outliers"]==True]

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff,MGMPI,outliers
9,2,Hardin,19,M,428,Week8,55,1.0,19.387388,True
22,5,Nancy,40,F,428,Week2,25,2.5,37.824910,True
29,6,George,53,M,506,Week8,45,1.0,42.800454,True


# 从身份证号提取出生年月并计算年龄

In [18]:
df2 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\cardsrecord.xlsx",sheet_name="Sheet1");

In [19]:
df2.head()

,ID,Name,CardID
0,1,张三,42010619830615XXXX
1,2,李四,42011119690326XXXX
2,3,王五,42010219640306XXXX
3,4,申四,42222519690227XXXX
4,5,谢五,42010719870319XXXX


In [20]:
df2.dtypes  # CardID 列是字符型的

ID         int64
Name      object
CardID    object
dtype: object

In [42]:
# 使用 series的apply方法，它可以接受一个函数，作用在此 series上。
# 此处的函数为 lambda 自定义函数，提取身份证号的第6到14位（从0开始，前闭后开），存储在新建列 Born 里。
# 并使用 pd.to_datetime 转化为 'Timestamp' 格式。

df2["Born"] = df2["CardID"].apply(lambda s: s[6:14])
df2["Born"] = pd.to_datetime(df2["Born"])

In [44]:
df2

,ID,Name,CardID,Born
0,1,张三,42010619830615XXXX,1983-06-15
1,2,李四,42011119690326XXXX,1969-03-26
2,3,王五,42010219640306XXXX,1964-03-06
3,4,申四,42222519690227XXXX,1969-02-27
4,5,谢五,42010719870319XXXX,1987-03-19
5,6,赵六,42230119841105XXXX,1984-11-05
6,7,唐七,42011619820729XXXX,1982-07-29
7,8,宋八,42010319900115XXXX,1990-01-15
8,9,钱九,42010719661023XXXX,1966-10-23
9,10,赵十,42010719640806XXXX,1964-08-06


In [48]:
# 使用 python 库 datetime 里的 datetime 模块。
# 计算出生年与今年的差，得到年龄。

from datetime import datetime
today = datetime.today().year  # 此为今天的年份
df2["Age"] = today - df2["Born"].dt.year  # dt.year 是一种方法 

In [49]:
df2

,ID,Name,CardID,Born,Age
0,1,张三,42010619830615XXXX,1983-06-15,36
1,2,李四,42011119690326XXXX,1969-03-26,50
2,3,王五,42010219640306XXXX,1964-03-06,55
3,4,申四,42222519690227XXXX,1969-02-27,50
4,5,谢五,42010719870319XXXX,1987-03-19,32
5,6,赵六,42230119841105XXXX,1984-11-05,35
6,7,唐七,42011619820729XXXX,1982-07-29,37
7,8,宋八,42010319900115XXXX,1990-01-15,29
8,9,钱九,42010719661023XXXX,1966-10-23,53
9,10,赵十,42010719640806XXXX,1964-08-06,55


# Calnan函数 计算df每一列的缺失值数目和比例

In [54]:
# Calnan 函数

# 定义了一个计算 缺失值数目的函数 Calnan。
# df 是待计算的 dataframe，本函数可以计算 df 的每一列的缺失值 nan 的数目。
# 返回的是一个 dataframe，有三列，第一列是 df的每个列名，第二列 nancount是 df每一列有多少个缺失值。
# nanpercent是 df 的每一列的缺失值占 df 总条目数的比例。

def Calnan(df):
    num = df.shape[0]     # df 的总条目数
    outcome = []
    for i in df.columns:  # 对 df 的每列进行循环 ！！！
        nancount = np.sum(df[i].isnull())   # series的 innull 方法对此series的每个元素是否缺失进行 True或 False的判断
                                            # 因为 True可以当做数值1使用，所以 sum函数可以计算 True的个数。
        nanpercent = nancount/num
        nanpercent = "%.2f%%" % (nanpercent * 100)  # 以百分数形式展示
        outcome.append([i, nancount, nanpercent])  # 循环得到的 outcome 是二维 ndarray
    return pd.DataFrame(outcome, columns=["column_name","nancount","nanpercent"])  # 将二维 ndarray 转化为 df 返回。


In [55]:
df3 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT2.xlsx",sheet_name="Sheet1");

In [56]:
Calnan(df3)

,column_name,nancount,nanpercent
0,ID,0,0.00%
1,Name,0,0.00%
2,Age,0,0.00%
3,Gender,0,0.00%
4,Treatment,0,0.00%
5,Phase,0,0.00%
6,Yeaple,7,11.67%
7,Schiff,8,13.33%
8,MGMPI,5,8.33%


# Calneg函数 计算df的每一列的负值个数和比例

In [58]:
# 定义了一个计算 负值比例的函数 Calneg。
# df 是待计算的 dataframe，注意，每一列必须是数值型列。本函数可以计算 df 的每一列的最负值个数、负值占此列值总个数
# （也就是df的总条目数）的比例。
# 返回的是一个 dataframe，有三列，第一列是 df的每个列名，第二列 是 负值个数，第三列是负值个数比例。



def Calvalues(df):
    num = df.shape[0]   # df 的总条目数
    outcome = []
    for i in df.columns:   # 对 df 的每列进行循环！！！
        negativevaluecount = df.loc[df[i]<0,:].shape[0]   # : 处应是：不应是 df[i]，否则报错。可能和格式有关系。
        negativevaluepercent = negativevaluecount/num
        negativevaluepercent = "%.2f%%" % (negativevaluepercent * 100)  # 以百分数形式展示
        outcome.append([i,negativevaluecount,negativevaluepercent])
    return pd.DataFrame(outcome,columns=["column_name","Negative_count","Negative_percent"])


In [59]:
df4 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT3.xlsx",sheet_name="Sheet1");

In [61]:
Calvalues(df4.iloc[:,6:9])

,column_name,Negative_count,Negative_percent
0,Yeaple,0,0.00%
1,Schiff,2,3.33%
2,MGMPI,1,1.67%


# 计算变量包含的值和个值出现的次数

In [143]:
df1.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff,MGMPI
0,1,Tracy,28,F,316,Base,10,3.0,73.356009
1,1,Tracy,28,F,316,Week1,15,2.5,66.468254
2,1,Tracy,28,F,316,Week2,25,1.0,64.980159
3,1,Tracy,28,F,316,Week6,55,0.5,83.524397
4,1,Tracy,28,F,316,Week8,60,0.5,73.469388


In [144]:
# series 的 value_counts() 方法可以计算一个数值型列中都包含什么值，以及此值出现的次数。 

df1["Schiff"].value_counts()

2.5    17
1.0    13
2.0    10
1.5     8
0.5     6
3.0     6
Name: Schiff, dtype: int64

# 计算变量的值是否是唯一无重复的

In [153]:
s1 =pd.Series([1,2,3,2,3,3])
s2 = pd.Series( [1,2,3])  
s1.is_unique

False

In [154]:
s2.is_unique

True

# 生成有规律的列名

In [62]:
# L1 是 临床试验的 phase 代号。L2 是 指标代号。 Desen 是 Desensitization 的简写（脱敏，即 抗敏感）
# L 是生成的 变量名列表，可以把它赋给 df 。

L1 = ['base','week1','week2','week4','week8','week12']
L2 = ['yeaple','schiff','VAS']
L = ['_'.join(['Desen',i,j]) for i in L1 for j in L2]    
L

['Desen_base_yeaple',
 'Desen_base_schiff',
 'Desen_base_VAS',
 'Desen_week1_yeaple',
 'Desen_week1_schiff',
 'Desen_week1_VAS',
 'Desen_week2_yeaple',
 'Desen_week2_schiff',
 'Desen_week2_VAS',
 'Desen_week4_yeaple',
 'Desen_week4_schiff',
 'Desen_week4_VAS',
 'Desen_week8_yeaple',
 'Desen_week8_schiff',
 'Desen_week8_VAS',
 'Desen_week12_yeaple',
 'Desen_week12_schiff',
 'Desen_week12_VAS']

# 变量类型转换

In [75]:
df1.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff,MGMPI
0,1,Tracy,28,F,316,Base,10,3.0,73.356009
1,1,Tracy,28,F,316,Week1,15,2.5,66.468254
2,1,Tracy,28,F,316,Week2,25,1.0,64.980159
3,1,Tracy,28,F,316,Week6,55,0.5,83.524397
4,1,Tracy,28,F,316,Week8,60,0.5,73.469388


In [76]:
df1.dtypes

ID             int64
Name          object
Age            int64
Gender        object
Treatment      int64
Phase         object
Yeaple         int64
Schiff       float64
MGMPI        float64
dtype: object

将连续数值变量转化为 有序类别变量/无序类别变量

In [82]:
df1["Yeaple"].astype("category", ordered=True)   # ordered=True 表示“有序” 若 ordered=False 表示无序

C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


0     10
1     15
2     25
3     55
4     60
5     15
6     20
7     30
8     50
9     55
10    15
11    15
12    20
13    30
14    40
15    10
16    20
17    30
18    50
19    60
20    10
21    15
22    25
23    40
24    50
25    10
26    20
27    25
28    35
29    45
30    15
31    20
32    30
33    45
34    55
35    15
36    20
37    30
38    45
39    60
40    10
41    20
42    25
43    45
44    50
45    10
46    20
47    35
48    60
49    60
50    15
51    25
52    35
53    55
54    55
55    10
56    15
57    25
58    45
59    50
Name: Yeaple, dtype: category
Categories (11, int64): [10 < 15 < 20 < 25 ... 45 < 50 < 55 < 60]

将字符串/数值转化为 日期变量

In [115]:
df5 = pd.DataFrame({"STR":["19901005","01/01/1980","28-06-1970"],"NUM":[19901005,19800101,19700628]})
df5

,NUM,STR
0,19901005,19901005
1,19800101,01/01/1980
2,19700628,28-06-1970


In [116]:
# 字符格式的变量可以直接用 pd.to_datetime 转化。
# 数值型变量需要先转化为 字符型，再用 pd.to_datetime 转化，否则出错。

df5["STR"] = pd.to_datetime(df5["STR"])
df5["NUM"] = pd.to_datetime(df5["NUM"].astype("str"))  # 这里一定要是 "str" 而非 "object"
df5

,NUM,STR
0,1990-10-05,1990-10-05
1,1980-01-01,1980-01-01
2,1970-06-28,1970-06-28


In [117]:
df5.dtypes

NUM    datetime64[ns]
STR    datetime64[ns]
dtype: object

# 随机分层划分数据集

In [124]:
df6 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT4.xlsx",sheet_name="Sheet1");
df6

,ID,Name,Age,Gender
0,1,Tracy,28,F
1,2,Hardin,19,M
2,3,Tessa,20,F
3,4,Neil,35,M
4,5,Nancy,40,F
5,6,George,53,M
6,7,Caroline,38,F
7,8,Zed,22,M
8,9,Steph,25,F
9,10,Landon,32,M


In [126]:
# 利用 sklearn 的 train_test_split 实现随机划分数据集。
# 注意，此划分可以实现 “随机” 和 “分层” 的功能。
# shuffle=True 实现随机划分， stratify=df6["Gender"] 指出分层的依据列。  test_size = 0.2 表明是 二八划分。
# 返回划分好的两份数据集。

from sklearn.model_selection import train_test_split
df6_80,df6_20 = train_test_split(df6, test_size = 0.2, shuffle=True, stratify=df6["Gender"], random_state = 0)

In [127]:
df6_80

,ID,Name,Age,Gender
1,2,Hardin,19,M
9,10,Landon,32,M
6,7,Caroline,38,F
7,8,Zed,22,M
8,9,Steph,25,F
4,5,Nancy,40,F
2,3,Tessa,20,F
3,4,Neil,35,M
5,6,George,53,M


In [128]:
df6_20

,ID,Name,Age,Gender
11,12,Ken,50,M
0,1,Tracy,28,F
10,11,Molly,46,F


# 计算相关系数矩阵

In [135]:
df8 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT5.xlsx",sheet_name="Sheet1");
df8

,ID,Name,SchiffMean,YeapleMean,MGMPIMean
0,1,Tracy,1.5,33,72.359641
1,2,Hardin,1.8,34,56.557324
2,3,Tessa,2.0,24,73.848189
3,4,Neil,1.5,34,66.254355
4,5,Nancy,2.1,28,66.085331
5,6,George,1.8,27,67.807802
6,7,Caroline,1.8,33,94.153518
7,8,Zed,1.7,34,76.433225
8,9,Steph,2.2,30,87.720791
9,10,Landon,2.0,35,84.733367


In [138]:
from sklearn.preprocessing import StandardScaler
from sklearn import covariance

In [139]:
scaler1 = StandardScaler()
df8_s = df8[["SchiffMean","YeapleMean","MGMPIMean"]]
df8_standard = scaler1.fit_transform(df8_s)

In [140]:
df8_standard

array([[-1.55750997,  0.40699407, -0.31776533],
       [-0.11980846,  0.67832345, -1.83787776],
       [ 0.83865922, -2.03497034, -0.17457367],
       [-1.55750997,  0.67832345, -0.90506665],
       [ 1.31789306, -0.94965283, -0.92132598],
       [-0.11980846, -1.22098221, -0.75563196],
       [-0.11980846,  0.40699407,  1.77870836],
       [-0.5990423 ,  0.67832345,  0.07409519],
       [ 1.79712689, -0.40699407,  1.1599087 ],
       [ 0.83865922,  0.94965283,  0.87253185],
       [-0.11980846,  1.49231158,  0.03998384],
       [-0.5990423 , -0.67832345,  0.9870134 ]])

In [142]:
# 对标准化后的数据计算经验相关系数方阵。因为一共三个变量，所以得到的是 3*3 的相关系数方阵。

df8_standard_correlation_matrix = covariance.empirical_covariance(df8_standard)
df8_standard_correlation_matrix

array([[ 1.        , -0.39550859,  0.23478307],
       [-0.39550859,  1.        ,  0.05697815],
       [ 0.23478307,  0.05697815,  1.        ]])

# 列出当前目录下的所有文件

In [155]:
import os                      # 导入os模块
[d for d in os.listdir('.')]   # os.listdir可以列出文件和目录 。其实导出的是 clinical trials 这个文件夹下的所有文件

['.ipynb_checkpoints', 'SASSQLPANDAS code.ipynb', 'Tools.ipynb']

# 规范化变量名

In [163]:
df12 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT6.xlsx",sheet_name="Sheet1");
df12.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff
0,1,tracy,28,F,316,Base,10,3.0
1,1,Tracy,28,F,316,Week1,15,2.5
2,1,tracy,28,F,316,Week2,25,1.0
3,1,TraCy,28,F,316,Week6,55,0.5
4,1,Tracy,28,F,316,Week8,60,0.5


In [165]:
# 把原始数据集中输入的不规范的英文名字，变为首字母大写，其他小写的规范名字。主要使用 字符串的 capitalize 方法


df12["Name"] = df12["Name"].apply(lambda s: s.capitalize())
df12.head()

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff
0,1,Tracy,28,F,316,Base,10,3.0
1,1,Tracy,28,F,316,Week1,15,2.5
2,1,Tracy,28,F,316,Week2,25,1.0
3,1,Tracy,28,F,316,Week6,55,0.5
4,1,Tracy,28,F,316,Week8,60,0.5


# 给 df 的列名加前缀

In [167]:
df12.head(2)

,ID,Name,Age,Gender,Treatment,Phase,Yeaple,Schiff
0,1,Tracy,28,F,316,Base,10,3.0
1,1,Tracy,28,F,316,Week1,15,2.5


In [168]:
# 设置df12的各个列名，给列名加上DEN_前缀。再赋值给 df12.columns

df12.columns = ["{}_{}".format('DEN_', i) for i in df12.columns]

In [169]:
df12.head(2)

,DEN__ID,DEN__Name,DEN__Age,DEN__Gender,DEN__Treatment,DEN__Phase,DEN__Yeaple,DEN__Schiff
0,1,Tracy,28,F,316,Base,10,3.0
1,1,Tracy,28,F,316,Week1,15,2.5


# 将日期转化为 行索引 并排序

In [173]:
df14 = pd.read_excel(r"F:\Clinical trials\SASSQLPANDAS\originTT7.xlsx",sheet_name="Sheet1");
df14["Time"] = pd.to_datetime(df14["Time"].astype("str"))   # 使用 astype 现将数值型 20180304 转化为字符串 '20180304'
df14.head()                                                 # 再使用 pd.to_datetime 方法

,ID,Name,Age,Gender,Time
0,1,Tracy,28,F,2018-03-21
1,2,Hardin,19,M,2018-03-25
2,3,Tessa,20,F,2018-04-06
3,4,Neil,35,M,2018-05-03
4,5,Nancy,40,F,2018-02-14


In [174]:
# 注意，使用 set_index 方法 设置 行索引，使用 sort_index 方法排序。

df14.set_index(["Time"]).sort_index()

,ID,Name,Age,Gender
Time,,,,
2018-02-04,11,Molly,46,F
2018-02-14,5,Nancy,40,F
2018-03-20,7,Caroline,38,F
2018-03-21,1,Tracy,28,F
2018-03-23,6,George,53,M
2018-03-25,2,Hardin,19,M
2018-04-01,8,Zed,22,M
2018-04-05,12,Ken,50,M
2018-04-06,3,Tessa,20,F


# pd.concat( )合并数据集

In [ ]:
'''
pd.concat()实现将多张表沿着某个轴连接（concatenate）“拼接”成一张表（上下/左右 拼接）。

1. 默认axis=0，表示上下连接。
   几张表的行索引是不会合并的，而列若有重复，则会合并。而对于每个表单独拥有的列（即非重复的列）的处理方法，
由join和join_axes决定。
join=”outer”表示取所有表的所有列名的并集，即每个表单独拥有的列都会保留下来，重复的列当然也保留下来。
join=’inner’ 表示取所有表的所有列名的交集，即每个表单独拥有的列都会舍弃，只保留那些重复列。
join_axes其实是自定义了要保留的列名，如join_axes=[‘A’, ‘B’, ‘C’]表示只保留A,B,C三列，join_axes=[df1.columns]
表示只保留表df1拥有的列（即leftjoin/rightjoin功能）。
ignore_index可以为这个大表重新赋行索引。keys参数设置在最外层添加一层行索引，指出每行来自哪张表。

2. 若axis=1，表示左右连接。
   几张表的列是不会合并的，而行索引若有重复，则会合并。而对于每个表单独拥有的行索引（即非重复的行索引）的处理方法，
由join和join_axes决定。
join=”outer”表示取所有表的所有行索引的并集，即每个表单独拥有的行索引都会保留下来，重复的行索引当然也保留下来。
join=’inner’ 表示取所有表的所有行索引的交集，即每个表单独拥有的行索引都会舍弃，只保留那些重复行索引。
join_axes其实是自定义了要保留的行索引，如join_axes=[0, 1,3]表示只保留行索引为0,1,3的三行。

'''

In [176]:
dfa = pd.DataFrame({"A":['A0','A1','A2','A3'],
                    "B":['B0','B1','B2','B3'],
                    "C":['C0','C1','C2','C3'],
                    "D":['D0','D1','D2','D3']}, index=[0,1,2,3])

dfb = pd.DataFrame({"A":['A4','A5','A6','A7'],
                    "B":['B4','B5','B6','B7'],
                    "C":['C4','C5','C6','C7'],
                    "D":['D4','D5','D6','D7']}, index=[4,5,6,7])

dfc = pd.DataFrame({"A":['A8','A9','A10','A11'],
                    "B":['B8','B9','B10','B11'],
                    "C":['C8','C9','C10','C11'],
                    "D":['D8','D9','D10','D11']}, index=[8,9,10,11])

dfd = pd.DataFrame({
                    "B":['B2','B3','B6','B7'],
                    "D":['D2','D3','D6','D7'],
                    "F":['F2','F3','F6','F7']}, index=[2,3,6,7])

In [177]:
dfa

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [178]:
dfb

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [179]:
dfc

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [180]:
dfd

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [181]:
# 上下连接

pd.concat([dfa, dfb, dfc])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [182]:
# 左右连接

pd.concat([dfa, dfb, dfc],axis=1)

,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,B8,C8,D8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A9,B9,C9,D9


In [183]:
# 上下连接，行索引重复的也不合并，但列重复的会合并。

pd.concat([dfa, dfd])

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [184]:
# 左右连接，列重复的不会合并，行索引重复的合并。

pd.concat([dfa, dfd], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


In [185]:
# 上下连接，行索引重复的不会合并，列重复的会合并，取列的交集展示。

pd.concat([dfa, dfd], join="inner")

,B,D
0,B0,D0
1,B1,D1
2,B2,D2
3,B3,D3
2,B2,D2
3,B3,D3
6,B6,D6
7,B7,D7


In [186]:
# 上下连接，行索引重复的不会合并，列重复的会合并，取dfa表里的列展示。

pd.concat([dfa, dfd], join_axes=[dfa.columns])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
2,NaN,B2,NaN,D2
3,NaN,B3,NaN,D3
6,NaN,B6,NaN,D6
7,NaN,B7,NaN,D7


In [188]:
# keys参数指明来源，源于哪张表。

pd.concat([dfa, dfd], keys=["x","y"])

A   B    C   D    F
x 0   A0  B0   C0  D0  NaN
  1   A1  B1   C1  D1  NaN
  2   A2  B2   C2  D2  NaN
  3   A3  B3   C3  D3  NaN
y 2  NaN  B2  NaN  D2   F2
  3  NaN  B3  NaN  D3   F3
  6  NaN  B6  NaN  D6   F6
  7  NaN  B7  NaN  D7   F7

# pd.merge( )合并数据集

In [ ]:
'''
    pd.merge() 函数每次只能组合两张表。
    有点类似SAS的merge a b; by id; 但有不同。
（SAS的Merge当一对多时还好，但多对一时会有意外的结果，多对一时和pandas的此merge
函数的效果就不一样了。）

    merge和concat的区别：concat是两张表可以上下连接，可以左右连接，用axis参数控制。
而merge函数只将两张表“左右融合”，而不进行“上下融合”。（merge并没有axis参数）。
concat只是将若干张表按某个方向“首尾拼接”在一起，而merge是将两张表“融合”在一起，
不太一样。

1. 参数left指定左表，即一个df表。right指定右表，另一个df表。

2. 通过参数on/ left_on, right_on/left_index, right_index指定左右两表各自的“依从列”
（可以是列名，也可以是行索引label（行索引是竖着的，类似列））。两表的依从列在结果表里
融合成了一列，融合的标准是“等值连接”（或说“等值匹配”），即若左表某行在其依从列上的
值=右表某行在其依从列上的值，则“等值匹配”，左表此行和右表此行通过依从列融合成一长行。
若一个表的某行与另一个表的多行都在依从列上“等值匹配”，则产生多个融合的长行（也就是说，
如果匹配，则产生这些匹配条目的笛卡尔集）。另外，两表各自的“依从列”，可以是一列，也可以
是多列，融合标准都是“等值匹配”。

3. 参数how设置依从列的“融合方式”，四值可选：{'left', 'right', 'outer', 'inner'},
默认为 'inner'。也就是，两表的依从列，可能有些值是匹配的，但有些值是各表依从列独有的，
在对方表里没有匹配，则它们怎么处理？'outer'表示取左右两表依从列值的并集，说明结果表里
保留两表各自独有的行，当然也保留匹配的长行。'inner'表示取左右两表依从列值的交集，说明
结果表里只保留匹配的长行，不保留两表各自独有的行。'left'表示产生左表的完全集，右表中
有匹配的就保留，右表独有的行就删掉。 'right'表示产生右表的完全集，左表中有匹配的就保留，
左表独有的行就删掉。

4. 我们知道，左右两表各自的依从列在结果表里融合成一列，那如果左右两表除依从列外还有
同名的列，怎么办？pandas的此merge函数不对它们进行处理，即不会合并它们（这和SAS的MERGE
函数不同，SAS的merge函数的同名列会发生覆盖），而是将它们都保留下来。但是，系统会给他们
自动加后缀_x, _y以让他们不再重名。你也可以通过设置suffixes参数来手动给同名列的列名加
后缀以改变列名，使其不再重名。

5. sort参数设置是否对结果表进行排序（排序依赖列即两表各自的依从列在结果表里对应的那一个
融合列）

'''

In [194]:
left = pd.DataFrame({"key1":['K0','K0','K1','K3'],
                     "key2":['K0','K1','K0','K1'],
                     "A":['A0','A1','A2','A3'],
                     "B":['B0','B1','B2','B3'],
                     })

right = pd.DataFrame({"key1":['K0','K1','K1','K2'],
                     "key2":['K0','K0','K0','K0'],
                     "C":['C0','C1','C2','C3'],
                     "D":['D0','D1','D2','D3'],
                     })

In [195]:
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K3,K1


In [196]:
right

,C,D,key1,key2
0,C0,D0,K0,K0
1,C1,D1,K1,K0
2,C2,D2,K1,K0
3,C3,D3,K2,K0


In [197]:
# 依照 key1 列左右融合。key1列取交集。

pd.merge(left,right,how="inner",on="key1")

,A,B,key1,key2_x,C,D,key2_y
0,A0,B0,K0,K0,C0,D0,K0
1,A1,B1,K0,K1,C0,D0,K0
2,A2,B2,K1,K0,C1,D1,K0
3,A2,B2,K1,K0,C2,D2,K0


In [198]:
# 依照 key1 列左右融合。key1列取并集

pd.merge(left,right,how="outer",on="key1")

,A,B,key1,key2_x,C,D,key2_y
0,A0,B0,K0,K0,C0,D0,K0
1,A1,B1,K0,K1,C0,D0,K0
2,A2,B2,K1,K0,C1,D1,K0
3,A2,B2,K1,K0,C2,D2,K0
4,A3,B3,K3,K1,NaN,NaN,NaN
5,NaN,NaN,K2,NaN,C3,D3,K0


In [199]:
# 依照 key1 列左右融合。key1列做数据集的值

pd.merge(left,right,how="left",on="key1")

,A,B,key1,key2_x,C,D,key2_y
0,A0,B0,K0,K0,C0,D0,K0
1,A1,B1,K0,K1,C0,D0,K0
2,A2,B2,K1,K0,C1,D1,K0
3,A2,B2,K1,K0,C2,D2,K0
4,A3,B3,K3,K1,NaN,NaN,NaN


In [201]:
# 依照 key1 列左右融合。suffixes参数自定义重名列（非融合依从列）的列名。

pd.merge(left,right,how="left",on="key1", suffixes=('_L', '_R'))

,A,B,key1,key2_L,C,D,key2_R
0,A0,B0,K0,K0,C0,D0,K0
1,A1,B1,K0,K1,C0,D0,K0
2,A2,B2,K1,K0,C1,D1,K0
3,A2,B2,K1,K0,C2,D2,K0
4,A3,B3,K3,K1,NaN,NaN,NaN
